<a href="https://colab.research.google.com/github/RubenGres/AerialDreams/blob/main/AerialDreams_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Using Drawing2Map directly with the diffusers API

In [ ]:
!pip install diffusers transformers git+https://github.com/huggingface/accelerate.git

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

controlnet = ControlNetModel.from_pretrained("rgres/sd-controlnet-aerialdreams", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base", controlnet=controlnet, torch_dtype=torch.float16
)

# CPU offloading for faster inference times
from diffusers import UniPCMultistepScheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

image = load_image("https://raw.githubusercontent.com/RubenGres/AerialDreams/main/validation/mask/MSK_076201.png")
image

In [ ]:
prompt = "High resolution image, 4K, ultra detailed, aerial view of 31 Rue Molière, France."

generator = torch.manual_seed(0)

images = pipe(
    prompt=[prompt]*4,
    num_inference_steps=20,
    image=image
)

for img in images[0]:
  img.show()

## 2. Using Drawing2Map with a simple gradio interface

In [ ]:
!git clone https://github.com/RubenGres/Drawing2Map-hfspace.git
%cd Drawing2Map-hfspace/ui
!pip install -r requirements.txt

from PIL import Image
from io import BytesIO
import base64

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import torch

import gradio as gr

controlnet = ControlNetModel.from_pretrained("rgres/sd-controlnet-aerialdreams", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base", controlnet=controlnet, torch_dtype=torch.float16
)

pipe = pipe.to("cuda")

# CPU offloading for faster inference times
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def generate_map(image, prompt, steps, seed):
    #image = Image.open(BytesIO(base64.b64decode(image_base64)))
    generator = torch.manual_seed(seed)

    image = Image.fromarray(image)

    image = pipe(
        prompt=prompt,
        num_inference_steps=steps,
        image=image
    ).images[0]

    return image

demo = gr.Interface(
  fn=generate_map,
  inputs=["image", "text", gr.Slider(0,100), "number"],
  outputs="image")

demo.launch(share=True)